In [ ]:
#default_exp filter
%load_ext autoreload
%autoreload 2

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#export
from pathlib import Path
import pandas as pd
from datetime import datetime
from anateldb.query import *
from anateldb.constants import console
from unidecode import unidecode
from fastcore.test import *
from fastcore.script import call_parse, Param, store_true, bool_arg
from pyarrow import ArrowInvalid

In [ ]:
#export
@call_parse
def formatar_db(
    path: Param("Pasta onde salvar os arquivos", str),
    up_stel: Param("Atualizar a base do Stel", bool_arg) = False, 
    up_radcom: Param("Atualizar a base do Radcom", bool_arg) = False,
    up_mosaico: Param("Atualizar a base do Mosaico", bool_arg) = False,
) -> None:
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    time = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    console.print(":scroll:[green]Lendo as bases de dados...")
    rd = read_base(path, up_stel, up_radcom, up_mosaico)
    rd['Validade_RF'] = rd.Validade_RF.astype('string').fillna('')
    rd['Data_Ato'] = rd.Data_Ato.astype('string').fillna('')
    rd['Status'] = rd.Status.astype('string')
    rd['Classe'] = rd.Classe.astype('string')
    rd.loc[rd['Status'] != '', 'Status'] = rd.loc[rd['Status'] != '', 'Status'] + ", " \
        + rd.loc[rd['Status'] != '', 'Classe']
    rd.loc[rd['Status'].isna(), 'Status'] = rd.loc[rd['Status'].isna(), 'Num_Serviço'].astype('string')

    rd["Descrição"] = (
        rd.Status
        + ", "
        + rd.Entidade.astype('string').str.title()
        + " ("
        + rd.Fistel.astype('string')
        + ", "
        + rd["Número_da_Estação"].astype('string')
        + "), "
        + rd.Município.astype('string')
        + "/"
        + rd.UF.astype('string')
    )

    export_columns = [
        "Frequência",
        "Latitude",
        "Longitude",
        "Descrição",
        "Num_Serviço",
        "Número_da_Estação",
        "Num_Ato",
        "Data_Ato",
        "Validade_RF",
    ]
    rd = rd.loc[:, export_columns]
    rd.columns = APP_ANALISE
    console.print(":card_file_box:[green]Salvando os arquivos...")
    date = pd.DataFrame(columns=[time])
    try:
        rd.to_feather(f"{dest}/AnatelDB.fth")
    except ArrowInvalid:
        with pd.ExcelWriter(f"{dest}/AnatelDB.xlsx") as workbook:
            date.to_excel(workbook, sheet_name="ExtractDate", index=False)
            rd.to_excel(workbook, sheet_name="DataBase", index=False)
    console.print("Sucesso :zap:")

In [ ]:
pasta = Path(r'G:\Meu Drive\repos\Code\BaseDados')

In [ ]:
formatar_db(pasta, True, True, True)

🌒  Lendo o Banco de Dados do STEL. Processo Lento, aguarde...

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
